In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from lightgbm import LGBMClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('does_it_work.csv')
pd.set_option('display.max_columns', 110)
df

,AGEP,CIT,DEYE,MAR,SEX,ANC,NATIVITY,RACAIAN,RACASN,RACBLK,...,HICOV,NP,TYPE,FINCP,HINCP,RACES,MIL_STAT,INDUSTRY,SOCP_recat,MAR_YEARS
0,26,5,2,Never Married,Female,1,2,0,0,0,...,1,1,3,0.0,0.0,White,Served,Education,OFF,0.0
1,21,1,2,Never Married,Male,1,1,0,0,0,...,1,1,3,0.0,0.0,White,Served,Entertainment,EAT,0.0
2,18,2,2,Never Married,Female,1,1,0,0,0,...,1,1,3,0.0,0.0,Other,Served,Entertainment,SAL,0.0
3,85,1,2,Widowed,Female,4,1,0,0,0,...,1,1,2,0.0,0.0,White,Served,None,None,68.0
4,19,1,2,Never Married,Female,2,1,0,0,0,...,1,1,3,0.0,0.0,White,Served,Entertainment,ENT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159294,36,1,2,Married,Female,1,1,0,0,0,...,1,3,1,220000.0,220000.0,White,Served,Information,CMM,7.0
159295,38,1,2,Married,Male,1,1,0,1,0,...,1,3,1,220000.0,220000.0,Asian,Served,Professional,CMM,7.0
159296,27,1,2,Never Married,Male,1,1,0,0,0,...,1,3,1,64000.0,64000.0,White,Served,Education,EDU,0.0
159297,56,4,2,Married,Male,4,2,0,0,0,...,1,3,1,64000.0,64000.0,White,Served,Transportation,TRN,28.0


In [4]:
df['HICOV'].value_counts()

1    151320
0      7979
Name: HICOV, dtype: int64

In [5]:
X = df.drop(columns=['HICOV'])
y = df['HICOV']

In [7]:
continuous_columns = ['AGEP','INTP_cat','OIP_cat','RETP_cat','SEMP_cat','SSIP_cat','JWAP_mins','JWDP_mins','TOTAL_commute','PWGTP_y','JWMNP','PAP','SSP','WAGP','WKHP','OCCP','PERNP','PINCP',
                     'FINCP','HINCP','MAR_YEARS','NP']
categorical_columns = [x for x in X.columns if x not in continuous_columns]

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


ohe = OneHotEncoder()
_X_train_ohe = ohe.fit_transform(X_train[categorical_columns]).toarray()
_X_train_ohe_df = pd.DataFrame(data = _X_train_ohe,
                        columns = ohe.get_feature_names(categorical_columns))

_X_test_ohe = ohe.transform(X_test[categorical_columns]).toarray()
_X_test_ohe_df = pd.DataFrame(data = _X_test_ohe,
                        columns = ohe.get_feature_names(categorical_columns))

In [9]:

scaler = StandardScaler()
_X_train_cont = scaler.fit_transform(X_train[continuous_columns])
_X_train_cont_df = pd.DataFrame(data = _X_train_cont,
                         columns = continuous_columns)

_X_test_cont = scaler.fit_transform(X_test[continuous_columns])
_X_test_cont_df = pd.DataFrame(data = _X_test_cont,
                         columns = continuous_columns)

In [10]:
_X_test_merged = _X_test_cont_df[continuous_columns].merge(_X_test_ohe_df, left_index=True, right_index=True)
_X_train_merged = _X_train_cont_df[continuous_columns].merge(_X_train_ohe_df, left_index=True, right_index=True)
_X_train_merged

,AGEP,INTP_cat,OIP_cat,RETP_cat,SEMP_cat,SSIP_cat,JWAP_mins,JWDP_mins,TOTAL_commute,PWGTP_y,JWMNP,PAP,SSP,WAGP,WKHP,OCCP,PERNP,PINCP,FINCP,HINCP,MAR_YEARS,NP,CIT_1,CIT_2,CIT_3,CIT_4,CIT_5,DEYE_1,DEYE_2,MAR_Divorced,MAR_Married,MAR_Never Married,MAR_Separated,MAR_Widowed,SEX_Female,SEX_Male,ANC_1,ANC_2,ANC_3,ANC_4,NATIVITY_1,NATIVITY_2,RACAIAN_0,RACAIAN_1,RACASN_0,RACASN_1,RACBLK_0,RACBLK_1,RACNH_0,RACNH_1,RACNUM_1,RACNUM_2,RACNUM_3,RACNUM_4,RACNUM_5,...,WAOB_7,WAOB_8,CIT_STATUS_0,CIT_STATUS_1,BORN_CIT_0,BORN_CIT_1,TYPE_1,TYPE_2,TYPE_3,RACES_American Indigenous,RACES_Asian,RACES_Black,RACES_More Than One Race,RACES_Other,RACES_White,MIL_STAT_Served,INDUSTRY_Administration,INDUSTRY_Education,INDUSTRY_Entertainment,INDUSTRY_Finance,INDUSTRY_Information,INDUSTRY_Manufacturing,INDUSTRY_Medical,INDUSTRY_Military,INDUSTRY_None,INDUSTRY_Professional,INDUSTRY_Retail,INDUSTRY_Service,INDUSTRY_Social Service,INDUSTRY_Transportation,INDUSTRY_Wholesale,SOCP_recat_BUS,SOCP_recat_CLN,SOCP_recat_CMM,SOCP_recat_CMS,SOCP_recat_EAT,SOCP_recat_EDU,SOCP_recat_ENG,SOCP_recat_ENT,SOCP_recat_EXT,SOCP_recat_FFF,SOCP_recat_FIN,SOCP_recat_HLS,SOCP_recat_LGL,SOCP_recat_MED,SOCP_recat_MGR,SOCP_recat_MIL,SOCP_recat_None,SOCP_recat_OFF,SOCP_recat_PRD,SOCP_recat_PRS,SOCP_recat_PRT,SOCP_recat_SAL,SOCP_recat_SCI,SOCP_recat_TRN
0,0.962404,0.319058,-0.135018,-0.245121,5.078202,-0.169988,0.248737,0.175238,0.895517,-0.468177,-0.746768,-0.09087,-0.487801,-0.521691,1.231343,0.666702,0.862692,0.793119,0.432761,0.284290,1.481046,-0.513751,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.486627,-0.125129,-0.135018,-0.245121,-0.111339,-0.169988,0.769205,0.739064,0.713803,0.167690,-0.804819,-0.09087,-0.257177,0.181585,0.513204,-0.219882,0.143615,0.017352,-0.728598,0.449802,2.000353,-0.513751,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.857560,-0.118910,-0.135018,-0.245121,-0.111339,-0.169988,0.557765,0.482779,1.077230,-0.701761,-0.688716,-0.09087,-0.487801,0.089853,0.752584,0.819194,0.055990,-0.084274,0.244934,0.097633,1.429115,0.051066,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.805138,0.407895,-0.135018,0.781440,-0.111339,-0.169988,0.996910,0.585293,4.893208,-0.870461,0.391044,-0.09087,-0.487801,0.861564,0.991963,-0.327691,0.793149,0.928878,-0.728598,0.088094,-0.959699,-1.078568,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.396598,-0.125129,0.433421,-0.245121,-0.111339,-0.169988,-0.954845,-0.935328,-0.739903,0.816535,1.122495,-0.09087,-0.487801,-0.294546,0.178073,0.418458,-0.311199,-0.391219,0.895289,0.743938,-0.959699,0.615883,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,

In [11]:
from imblearn.over_sampling import SMOTE
_smote = SMOTE(sampling_strategy='minority', random_state=42)
_X_train_merged, y_train = _smote.fit_sample(_X_train_merged, y_train)

Using TensorFlow backend.


In [12]:
from sklearn.linear_model import LogisticRegression
bc_lr1 =LogisticRegression(solver='liblinear', penalty='l1')

# {'lr_C': 0.04108295837286748, 'lr_solver': 'saga'}
bc_lr1.fit(_X_train_merged, y_train)
y_train_pred = bc_lr1.predict(_X_train_merged)
y_test_pred = bc_lr1.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.8100724624905234
Testing Precision:  0.9834728390843868



Training Recall:  0.7593584213396797
Testing Recall:  0.7598825315119119



Training Accuracy:  0.7906605847257342
Testing Accuracy:  0.7594475831763967



Training F1-Score:  0.7838960640131029
Testing F1-Score:  0.8573396373925022


In [45]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=0.2)
from sklearn.linear_model import LogisticRegression




from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier(max_depth=5,
                            max_leaf_nodes=39,
                            max_features = .2479,
                            min_samples_leaf=8,
                            max_samples = 0.2647,
                            criterion='entropy')


# {'rf_max_depth': 5, 'rf_max_leaf_nodes': 39, 'rf_max_features': 0.24795749696390704, 'rf_min_samples_leaf': 8, 'rf_max_samples': 0.26471291186578816, 'rf_criterion': 'entropy'}
# parameters = {'max_leaf_nodes': range(10,100,1)}
#     'min_samples_split': range(2,10,1)}
#     'n_estimators': range(100, 500, 50)}
#     'max_depth': range(10,25,1)}
#              'min_samples_leaf': range(1,10,1),
#              'class_weight': ['balanced',None,'balanced_subsample']}
# grid_tree = GridSearchCV(rfc, parameters, cv=5, scoring='f1', n_jobs=-1,verbose=1)

rfc.fit(_X_train_merged, y_train)
y_train_pred = rfc.predict(_X_train_merged)
y_test_pred = rfc.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.8583934372075098
Testing Precision:  0.9775985288586116



Training Recall:  0.7730179979175963
Testing Recall:  0.77182736091863



Training Accuracy:  0.8227477812484506
Testing Accuracy:  0.7661330822347772



Training F1-Score:  0.8134717729312324
Testing F1-Score:  0.8626113252078993


In [240]:
from sklearn.linear_model import LogisticRegression
bc_lr1 =LogisticRegression(solver='liblinear', penalty='l1')

# {'lr_C': 0.04108295837286748, 'lr_solver': 'saga'}
bc_lr1.fit(_X_train_merged, y_train)
y_train_pred = bc_lr1.predict(_X_train_merged)
y_test_pred = bc_lr1.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.810049365303244
Testing Precision:  0.9834308408421233



Training Recall:  0.759350157832978
Testing Recall:  0.7598825315119119



Training Accuracy:  0.7906440577123308
Testing Accuracy:  0.7594161958568738



Training F1-Score:  0.7838808465626518
Testing F1-Score:  0.8573236788712469


In [241]:
bc_lr1.coef_

array([[ 1.80582822e-01, -5.35012125e-02,  1.32199286e-02,
         4.11188833e-01, -9.21189357e-02,  3.11613871e+00,
        -4.11432366e-02,  7.78033269e-02,  2.16947315e-01,
         7.18522892e-03, -4.53529677e-01,  1.01494870e-01,
         1.73130374e+00,  1.43466870e-01, -1.84579485e-02,
         8.63401639e-02,  8.01256775e-05,  1.80628569e-01,
         1.25705988e-01,  2.71404920e-01,  2.99790999e-01,
         6.63801364e-02, -4.86625809e-02,  0.00000000e+00,
         1.12142757e-01,  3.83786676e-01,  0.00000000e+00,
        -1.85637956e-01,  5.48190543e-02, -8.32838492e-02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.36162582e-01,  0.00000000e+00,
         5.03862307e-02,  2.01554754e-01, -2.78319851e-02,
         0.00000000e+00,  2.62641314e-01,  0.00000000e+00,
         0.00000000e+00,  8.09554774e-02, -3.37092310e-02,
         4.50763152e-02,  7.39304935e-02,  0.00000000e+00,
         3.83424928e-02, -1.54810276e+00,  0.00000000e+0

In [249]:
_coef_df = pd.DataFrame()
_coef_df['features'] = _X_train_merged.columns
_coef_df['coef'] = bc_lr1.coef_[0]
_coef_df

,features,coef
0,AGEP,0.180583
1,INTP_cat,-0.053501
2,OIP_cat,0.013220
3,RETP_cat,0.411189
4,SEMP_cat,-0.092119
...,...,...
461,SOCP_recat_PRS,0.207660
462,SOCP_recat_PRT,0.293448
463,SOCP_recat_SAL,-0.036184
464,SOCP_recat_SCI,1.710255


In [262]:
len(_selected)

369

In [258]:
_selected = _coef_df.loc[_coef_df['coef'] != 0]['features'].values

In [263]:
_coef_df.loc[_coef_df['coef'] == 0]['features'].values

array(['CIT_2', 'CIT_5', 'MAR_Married', 'MAR_Never Married',
       'MAR_Separated', 'MAR_Widowed', 'SEX_Male', 'ANC_4', 'NATIVITY_2',
       'RACAIAN_0', 'RACBLK_1', 'RACNUM_1', 'RACNUM_2', 'RACNUM_5',
       'RACPI_0', 'DOUT_cat_No', 'MARHW_cat_2.0', 'MARHW_cat_Unknown',
       'SCHL_cat_Grade_1', 'SCHL_cat_Grade_7', 'SCHL_cat_HS_Diploma',
       'WKW_cat_27_39wks', 'WKW_cat_Not_Worked', 'DRIVESP_cat_3_driver',
       'DRIVESP_cat_No_Drive', 'ESR_cat_ArmedForces_not_work',
       'NAICSP_cat_ADM', 'NAICSP_cat_ENT1', 'NAICSP_cat_ENT2',
       'NAICSP_cat_FIN1', 'NAICSP_cat_INF', 'NAICSP_cat_MED',
       'NAICSP_cat_No_Work', 'POBP_cat_0', 'COW_1.0', 'COW_10.0',
       'DDRS_2.0', 'DPHY_2.0', 'DREM_2.0', 'ENG_Not at All', 'JWRIP_6.0',
       'JWRIP_8.0', 'JWRIP_10.0', 'JWTR_1.0', 'JWTR_9.0', 'JWTR_10.0',
       'MARHM_2.0', 'MARHM_3.0', 'MARHT_2.0', 'MARHT_3.0', 'RELP_12',
       'DECADE_0.0', 'DECADE_1.0', 'DECADE_4.0', 'DIS_Not Disabled',
       'ESP_0.0', 'HISP_6', 'HISP_20', 'HISP_

In [250]:
_coef_df.to_csv('logreg_lasso_coef.csv', index=False, header=True)

In [248]:
len(bc_lr1.coef_[0])

466

In [253]:
from sklearn.ensemble import RandomForestClassifier

In [259]:
import optuna

def objective(trial):
    rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32)
    rf_max_leaf_nodes = trial.suggest_int('rf_max_leaf_nodes', 10, 64)
    rf_max_features = trial.suggest_loguniform('rf_max_features', .1, .9)
    rf_min_samples_leaf = trial.suggest_int('rf_min_samples_leaf', 2, 20)
    rf_max_samples = trial.suggest_loguniform('rf_max_samples', .1, .9)
    rf_criterion = trial.suggest_categorical('rf_criterion', ['gini', 'entropy'])
    
    regressor_obj = RandomForestClassifier(max_depth = rf_max_depth,
                                          max_leaf_nodes = rf_max_leaf_nodes,
                                          max_features = rf_max_features,
                                          min_samples_leaf = rf_min_samples_leaf,
                                          max_samples = rf_max_samples,
                                          n_estimators=300,
                                          random_state=42,
                                          criterion = rf_criterion)
    
    regressor_obj.fit(_X_train_merged[_selected], y_train)
    y_pred = regressor_obj.predict(_X_test_merged[_selected])
    
    return (1 - recall_score(y_test, y_pred))

In [260]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2020-08-19 16:51:22,220] Finished trial#0 with value: 0.1020260014518577 with parameters: {'rf_max_depth': 17, 'rf_max_leaf_nodes': 63, 'rf_max_features': 0.2427592357699389, 'rf_min_samples_leaf': 15, 'rf_max_samples': 0.11375335472398798, 'rf_criterion': 'gini'}. Best is trial#0 with value: 0.1020260014518577.
[I 2020-08-19 16:53:09,320] Finished trial#1 with value: 0.11225499901009706 with parameters: {'rf_max_depth': 27, 'rf_max_leaf_nodes': 55, 'rf_max_features': 0.1335350229490918, 'rf_min_samples_leaf': 5, 'rf_max_samples': 0.1403653433774214, 'rf_criterion': 'gini'}. Best is trial#0 with value: 0.1020260014518577.
[I 2020-08-19 16:55:50,055] Finished trial#2 with value: 0.2550980003959612 with parameters: {'rf_max_depth': 5, 'rf_max_leaf_nodes': 37, 'rf_max_features': 0.11688306002715892, 'rf_min_samples_leaf': 15, 'rf_max_samples': 0.49095100288265914, 'rf_criterion': 'entropy'}. Best is trial#0 with value: 0.1020260014518577.
[I 2020-08-19 16:57:09,498] Finished trial#3 wi

KeyboardInterrupt: 

In [261]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=0.2)
from sklearn.linear_model import LogisticRegression




from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier(max_depth=18,
                            max_leaf_nodes=64,
                            max_features = .3308894632219787,
                            min_samples_leaf=10,
                            max_samples = 0.10496753020075641,
                            criterion='gini')

#{'rf_max_depth': 18, 'rf_max_leaf_nodes': 64, 'rf_max_features': 0.3308894632219787, 'rf_min_samples_leaf': 10, 'rf_max_samples': 0.10496753020075641, 'rf_criterion': 'gini'}
# {'rf_max_depth': 5, 'rf_max_leaf_nodes': 39, 'rf_max_features': 0.24795749696390704, 'rf_min_samples_leaf': 8, 'rf_max_samples': 0.26471291186578816, 'rf_criterion': 'entropy'}
# parameters = {'max_leaf_nodes': range(10,100,1)}
#     'min_samples_split': range(2,10,1)}
#     'n_estimators': range(100, 500, 50)}
#     'max_depth': range(10,25,1)}
#              'min_samples_leaf': range(1,10,1),
#              'class_weight': ['balanced',None,'balanced_subsample']}
# grid_tree = GridSearchCV(rfc, parameters, cv=5, scoring='f1', n_jobs=-1,verbose=1)

rfc.fit(_X_train_merged, y_train)
y_train_pred = rfc.predict(_X_train_merged)
y_test_pred = rfc.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.8993576191914328
Testing Precision:  0.9720200664170141



Training Recall:  0.9104979589138447
Testing Recall:  0.9078730284432126



Training Accuracy:  0.9043044606409176
Testing Accuracy:  0.8875078468298807



Training F1-Score:  0.9048935025685038
Testing F1-Score:  0.9388521121954548


In [266]:
import optuna

def objective(trial):
    rf_max_depth = trial.suggest_int('rf_max_depth', 18, 54)
    rf_max_leaf_nodes = trial.suggest_int('rf_max_leaf_nodes', 54, 96)
    rf_max_features = trial.suggest_loguniform('rf_max_features', .1, .9)
    rf_min_samples_leaf = trial.suggest_int('rf_min_samples_leaf', 2, 20)
    rf_max_samples = trial.suggest_loguniform('rf_max_samples', .1, .9)
    rf_criterion = trial.suggest_categorical('rf_criterion', ['gini', 'entropy'])
    
    regressor_obj = RandomForestClassifier(max_depth = rf_max_depth,
                                          max_leaf_nodes = rf_max_leaf_nodes,
                                          max_features = rf_max_features,
                                          min_samples_leaf = rf_min_samples_leaf,
                                          max_samples = rf_max_samples,
                                          n_estimators=300,
                                          random_state=42,
                                          criterion = rf_criterion)
    
    regressor_obj.fit(_X_train_merged, y_train)
    y_pred = regressor_obj.predict(_X_test_merged)
    
    return (1 - recall_score(y_test, y_pred))

In [267]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2020-08-19 18:14:14,617] Finished trial#0 with value: 0.085956576255527 with parameters: {'rf_max_depth': 54, 'rf_max_leaf_nodes': 85, 'rf_max_features': 0.2011576096628556, 'rf_min_samples_leaf': 12, 'rf_max_samples': 0.5720394544994472, 'rf_criterion': 'gini'}. Best is trial#0 with value: 0.085956576255527.
[I 2020-08-19 18:25:57,824] Finished trial#1 with value: 0.08011614861743555 with parameters: {'rf_max_depth': 27, 'rf_max_leaf_nodes': 81, 'rf_max_features': 0.40431566940142366, 'rf_min_samples_leaf': 14, 'rf_max_samples': 0.24057939625788344, 'rf_criterion': 'entropy'}. Best is trial#1 with value: 0.08011614861743555.
[I 2020-08-19 18:36:23,550] Finished trial#2 with value: 0.08150201280274538 with parameters: {'rf_max_depth': 20, 'rf_max_leaf_nodes': 85, 'rf_max_features': 0.36469286613520613, 'rf_min_samples_leaf': 12, 'rf_max_samples': 0.2347469259526752, 'rf_criterion': 'entropy'}. Best is trial#1 with value: 0.08011614861743555.
[I 2020-08-19 18:50:49,448] Finished tria

KeyboardInterrupt: 

In [268]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=0.2)
from sklearn.linear_model import LogisticRegression




from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier(max_depth=27,
                            max_leaf_nodes=81,
                            max_features = .40431566940142366,
                            min_samples_leaf=14,
                            max_samples = 0.24057939625788344,
                            criterion='entropy')

# {'rf_max_depth': 27, 'rf_max_leaf_nodes': 81, 'rf_max_features': 0.40431566940142366, 'rf_min_samples_leaf': 14, 'rf_max_samples': 0.24057939625788344, 'rf_criterion': 'entropy'}
#{'rf_max_depth': 18, 'rf_max_leaf_nodes': 64, 'rf_max_features': 0.3308894632219787, 'rf_min_samples_leaf': 10, 'rf_max_samples': 0.10496753020075641, 'rf_criterion': 'gini'}
# {'rf_max_depth': 5, 'rf_max_leaf_nodes': 39, 'rf_max_features': 0.24795749696390704, 'rf_min_samples_leaf': 8, 'rf_max_samples': 0.26471291186578816, 'rf_criterion': 'entropy'}
# parameters = {'max_leaf_nodes': range(10,100,1)}
#     'min_samples_split': range(2,10,1)}
#     'n_estimators': range(100, 500, 50)}
#     'max_depth': range(10,25,1)}
#              'min_samples_leaf': range(1,10,1),
#              'class_weight': ['balanced',None,'balanced_subsample']}
# grid_tree = GridSearchCV(rfc, parameters, cv=5, scoring='f1', n_jobs=-1,verbose=1)

rfc.fit(_X_train_merged, y_train)
y_train_pred = rfc.predict(_X_train_merged)
y_test_pred = rfc.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.9053052744345342
Testing Precision:  0.9713697147445969



Training Recall:  0.9217859090683722
Testing Recall:  0.9180030357025012



Training Accuracy:  0.9126836564364453
Testing Accuracy:  0.8962649089767734



Training F1-Score:  0.9134712628617989
Testing F1-Score:  0.9439326852935688


In [296]:
# _X_train_merged

In [297]:
train_data = lgbm.Dataset(data=_X_train_merged, label=y_train, categorical_feature=ohe.get_feature_names(categorical_columns), free_raw_data=False)

# Create an LGBM dataset from the test
test_data = lgbm.Dataset(data=_X_test_merged, label=y_test, categorical_feature=ohe.get_feature_names(categorical_columns), free_raw_data=False)
lgbm_params = {
    'boosting': 'dart',          # dart (drop out trees) often performs better
    'application': 'binary',     # Binary classification
    'learning_rate': 0.05,       # Learning rate, controls size of a gradient descent step
    'min_data_in_leaf': 20,      # Data set is quite small so reduce this a bit
    'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
    'num_leaves': 41,            # Controls size of tree since LGBM uses leaf wise splits
    'metric': 'binary_logloss',  # Area under ROC curve as the evaulation metric
    'drop_rate': 0.15
              }

In [298]:
# list(ohe.get_feature_names(categorical_columns))

In [303]:
evaluation_results = {}
clf = LGBMClassifier(objective='binary',
                     boosting='gbdt',
                     learning_rate = 0.05,
                     max_depth = 8,
                     num_leaves = 80,
                     n_estimators = 400,
                     bagging_fraction = 0.8,
                     feature_fraction = 0.9)


clf.fit(_X_train_merged, y_train)
y_train_pred = clf.predict(_X_train_merged)
y_test_pred = clf.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.9625489946145757
Testing Precision:  0.9570546318289787



Training Recall:  0.9984216702199745
Testing Recall:  0.9971292813304297



Training Accuracy:  0.961613007007274
Testing Accuracy:  0.9547080979284369



Training F1-Score:  0.9801572171429962
Testing F1-Score:  0.9766810491103893


In [305]:
import pickle
with open('lgbm_classifier.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [306]:
clf

LGBMClassifier(bagging_fraction=0.8, boosting='gbdt', boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.05, max_depth=8,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=-1, num_leaves=80, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [302]:
evaluation_results = {}
clf = LGBMClassifier(
    objective='binary',
    boosting='gbdt',
    learning_rate = 0.05,
    max_depth = 14,
    num_leaves = 80,
    n_estimators = 400,
    bagging_fraction = 0.8,
    feature_fraction = 0.9)


clf.fit(_X_train_merged, y_train)
y_train_pred = clf.predict(_X_train_merged)
y_test_pred = clf.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.9673083850310186
Testing Precision:  0.9576308784383318



Training Recall:  0.9985704133406053
Testing Recall:  0.9971292813304297



Training Accuracy:  0.9665957830805326
Testing Accuracy:  0.9553044569993723



Training F1-Score:  0.9826908298399196
Testing F1-Score:  0.9769810222753871


In [ ]:
import optuna

def objective(trial):
    rf_max_depth = trial.suggest_int('rf_max_depth', 18, 54)
    rf_max_leaf_nodes = trial.suggest_int('rf_max_leaf_nodes', 54, 96)
    rf_max_features = trial.suggest_loguniform('rf_max_features', .1, .9)
    rf_min_samples_leaf = trial.suggest_int('rf_min_samples_leaf', 2, 20)
    rf_max_samples = trial.suggest_loguniform('rf_max_samples', .1, .9)
    rf_criterion = trial.suggest_categorical('rf_criterion', ['gini', 'entropy'])
    
    regressor_obj = RandomForestClassifier(max_depth = rf_max_depth,
                                          max_leaf_nodes = rf_max_leaf_nodes,
                                          max_features = rf_max_features,
                                          min_samples_leaf = rf_min_samples_leaf,
                                          max_samples = rf_max_samples,
                                          n_estimators=300,
                                          random_state=42,
                                          criterion = rf_criterion)
    
    regressor_obj.fit(_X_train_merged, y_train)
    y_pred = regressor_obj.predict(_X_test_merged)
    
    return (1 - recall_score(y_test, y_pred))

In [14]:
_X = X.copy()
_y = y.copy()

In [313]:
ohe_final = OneHotEncoder()
_X_final_ohe = ohe_final.fit_transform(_X[categorical_columns]).toarray()
_X_final_ohe_df = pd.DataFrame(data = _X_final_ohe,
                        columns = ohe_final.get_feature_names(categorical_columns))

In [314]:
from sklearn.preprocessing import StandardScaler
scaler_final = StandardScaler()
_X_final_cont = scaler_final.fit_transform(_X[continuous_columns])
_X_final_cont_df = pd.DataFrame(data = _X_final_cont,
                         columns = continuous_columns)


In [315]:
# _X_test_merged = _X_test_cont_df[continuous_columns].merge(_X_test_ohe_df, left_index=True, right_index=True)
_X_final_merged = _X_final_cont_df[continuous_columns].merge(_X_final_ohe_df, left_index=True, right_index=True)
_X_final_merged

,AGEP,INTP_cat,OIP_cat,RETP_cat,SEMP_cat,SSIP_cat,JWAP_mins,JWDP_mins,TOTAL_commute,PWGTP_y,JWMNP,PAP,SSP,WAGP,WKHP,OCCP,PERNP,PINCP,FINCP,HINCP,MAR_YEARS,NP,CIT_1,CIT_2,CIT_3,CIT_4,CIT_5,DEYE_1,DEYE_2,MAR_Divorced,MAR_Married,MAR_Never Married,MAR_Separated,MAR_Widowed,SEX_Female,SEX_Male,ANC_1,ANC_2,ANC_3,ANC_4,NATIVITY_1,NATIVITY_2,RACAIAN_0,RACAIAN_1,RACASN_0,RACASN_1,RACBLK_0,RACBLK_1,RACNH_0,RACNH_1,RACNUM_1,RACNUM_2,RACNUM_3,RACNUM_4,RACNUM_5,...,WAOB_7,WAOB_8,CIT_STATUS_0,CIT_STATUS_1,BORN_CIT_0,BORN_CIT_1,RACES_American Indigenous,RACES_Asian,RACES_Black,RACES_More Than One Race,RACES_Other,RACES_White,MIL_STAT_Served,TYPE_1,TYPE_2,TYPE_3,INDUSTRY_Administration,INDUSTRY_Education,INDUSTRY_Entertainment,INDUSTRY_Finance,INDUSTRY_Information,INDUSTRY_Manufacturing,INDUSTRY_Medical,INDUSTRY_Military,INDUSTRY_None,INDUSTRY_Professional,INDUSTRY_Retail,INDUSTRY_Service,INDUSTRY_Social Service,INDUSTRY_Transportation,INDUSTRY_Wholesale,SOCP_recat_BUS,SOCP_recat_CLN,SOCP_recat_CMM,SOCP_recat_CMS,SOCP_recat_EAT,SOCP_recat_EDU,SOCP_recat_ENG,SOCP_recat_ENT,SOCP_recat_EXT,SOCP_recat_FFF,SOCP_recat_FIN,SOCP_recat_HLS,SOCP_recat_LGL,SOCP_recat_MED,SOCP_recat_MGR,SOCP_recat_MIL,SOCP_recat_None,SOCP_recat_OFF,SOCP_recat_PRD,SOCP_recat_PRS,SOCP_recat_PRT,SOCP_recat_SAL,SOCP_recat_SCI,SOCP_recat_TRN
0,-1.239857,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,-0.955794,-0.936232,-0.740011,0.619286,1.123548,-0.090197,-0.486672,-0.448546,1.230394,0.772516,-0.457984,-0.564065,-0.730566,-0.873149,-0.959950,-1.078661,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.502367,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,3.677445,3.879619,-0.196193,-0.701801,-1.187609,-0.090197,-0.486672,-0.510080,-1.068971,0.417765,-0.516662,-0.618578,-0.730566,-0.873149,-0.959950,-1.078661,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.659874,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,-0.955794,-0.936232,-0.740011,-0.118968,1.123548,-0.090197,-0.486672,-0.524730,-1.164778,0.634163,-0.530633,-0.631557,-0.730566,-0.873149,-0.959950,-1.078661,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.857764,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,-0.955794,-0.936232,-0.740011,0.153020,1.123548,-0.090197,1.372175,-0.524730,-1.164778,-1.039910,-0.530633,-0.453740,-0.730566,-0.873149,2.571079,-1.078661,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.607372,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,2.019233,2.171870,-0.558738,-0.183727,-1.152767,-0.090197,-0.486672,-0.511984,-0.973165,-0.074276,-0.518478,-0.620265,-0.730566,-0.873149,-0.959950,-1.078661,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0

In [316]:
from imblearn.over_sampling import SMOTE
_smote_final = SMOTE(sampling_strategy='minority', random_state=42)
_X_final_merged, _y = _smote_final.fit_sample(_X_final_merged, _y)

In [317]:
evaluation_results = {}
clf_final = LGBMClassifier(objective='binary',
                     boosting='gbdt',
                     learning_rate = 0.05,
                     max_depth = 8,
                     num_leaves = 80,
                     n_estimators = 400,
                     bagging_fraction = 0.8,
                     feature_fraction = 0.9)


clf_final.fit(_X_final_merged, _y)
y_final_pred = clf_final.predict(_X_final_merged)
# y_test_pred = clf.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(_y, y_final_pred))
# print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(_y, y_final_pred))
# print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(_y, y_final_pred))
# print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(_y, y_final_pred))
# print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.9606139477914721



Training Recall:  0.9951229183187946



Training Accuracy:  0.9771609833465503



Training F1-Score:  0.9775639777197835


In [318]:
import pickle
with open('lgbm_final_classifier.pickle', 'wb') as f:
    pickle.dump(clf_final, f)

In [ ]:
ohe_final

In [319]:
with open('ohe_final.pickle', 'wb') as f:
    pickle.dump(ohe_final, f)

In [ ]:
scaler_final

In [320]:
with open('scaler_final.pickle', 'wb') as f:
    pickle.dump(scaler_final, f)

In [239]:
# list(_X_train_merged.columns)

In [237]:
select_columns = ['AGEP',
#  'INTP_cat',
 'OIP_cat',
 'RETP_cat',
 'SEMP_cat',
 'SSIP_cat',
#  'JWAP_mins',
 'JWDP_mins',
#  'TOTAL_commute',
 'PWGTP_y',
#  'JWMNP',
 'PAP',
 'SSP',
 'WAGP',
#  'WKHP',
 'OCCP',
#  'PERNP',
#  'PINCP',
 'FINCP',
#  'HINCP',
 'MAR_YEARS',
 'NP',
 'CIT_1',
 'CIT_2',
 'CIT_3',
 'CIT_4',
 'CIT_5',
 'DEYE_1',
#  'DEYE_2',
 'MAR_Divorced',
 'MAR_Married',
#  'MAR_Never Married',
 'MAR_Separated',
 'MAR_Widowed',
 'SEX_Female',
#  'SEX_Male',
 'ANC_1',
 'ANC_2',
#  'ANC_3',
#  'ANC_4',
#  'NATIVITY_1',
#  'NATIVITY_2',
#  'RACAIAN_0',
#  'RACAIAN_1',
 'RACASN_0',
#  'RACASN_1',
 'RACBLK_0',
#  'RACBLK_1',
#  'RACNH_0',
#  'RACNH_1',
 'RACNUM_1',
#  'RACNUM_2',
#  'RACNUM_3',
 'RACNUM_4',
#  'RACNUM_5',
 'RACPI_0',
#  'RACPI_1',
#  'RACSOR_0',
 'RACSOR_1',
 'DOUT_cat_No',
#  'DOUT_cat_Yes',
 'FER_cat_No',
#  'FER_cat_Not_Applicable',
 'FER_cat_Yes',
 'LANX_cat_English_Lang',
#  'LANX_cat_Other_Lang',
 'MARHW_cat_1.0',
#  'MARHW_cat_2.0',
#  'MARHW_cat_Unknown',
#  'MIL_cat_Active',
 'MIL_cat_Never Served',
#  'MIL_cat_Past Active',
 'MIL_cat_Reserves',
#  'MLPA_cat_LT_17yrold',
 'MLPA_cat_No_Served',
#  'MLPA_cat_Yes_Served',
 'SCHG_cat_Grade_10',
 'SCHG_cat_Grade_11',
 'SCHG_cat_Grade_12',
#  'SCHG_cat_Grade_6',
#  'SCHG_cat_Grade_7',
#  'SCHG_cat_Grade_8',
#  'SCHG_cat_Grade_9',
 'SCHG_cat_Graduate',
 'SCHG_cat_Not_Attending',
 'SCHG_cat_Undergrad',
 'SCHL_cat_Associates',
 'SCHL_cat_Bachelors',
 'SCHL_cat_Doctorate',
 'SCHL_cat_GED_alt',
#  'SCHL_cat_Grade_1',
 'SCHL_cat_Grade_10',
 'SCHL_cat_Grade_11',
 'SCHL_cat_Grade_12',
#  'SCHL_cat_Grade_2',
#  'SCHL_cat_Grade_3',
#  'SCHL_cat_Grade_4',
 'SCHL_cat_Grade_5',
 'SCHL_cat_Grade_6',
 'SCHL_cat_Grade_7',
 'SCHL_cat_Grade_8',
 'SCHL_cat_Grade_9',
 'SCHL_cat_HS_Diploma',
#  'SCHL_cat_Kindergarten',
 'SCHL_cat_LT1_College',
 'SCHL_cat_Masters',
 'SCHL_cat_NoDegree_College',
 'SCHL_cat_No_School',
#  'SCHL_cat_Nursery',
 'SCHL_cat_Professional',
 'WKL_cat_BT1_5',
#  'WKL_cat_GT5',
#  'WKL_cat_LT1',
 'WKW_cat_14_26wks',
#  'WKW_cat_27_39wks',
 'WKW_cat_40_47wks',
 'WKW_cat_48_49wks',
 'WKW_cat_50_52wks',
 'WKW_cat_LT_14wks',
#  'WKW_cat_Not_Worked',
#  'WRK_cat_Not_Reported',
 'WRK_cat_Not_Worked',
 'WRK_cat_Worked',
#  'YOEP_cat_Below_50s',
 'YOEP_cat_Immigr_00s',
 'YOEP_cat_Immigr_10s',
#  'YOEP_cat_Immigr_50s',
#  'YOEP_cat_Immigr_60s',
 'YOEP_cat_Immigr_70s',
 'YOEP_cat_Immigr_80s',
 'YOEP_cat_Immigr_90s',
#  'YOEP_cat_Naturalized',
 'DRIVESP_cat_1_driver',
 'DRIVESP_cat_2_driver',
#  'DRIVESP_cat_3_driver',
#  'DRIVESP_cat_4_driver',
#  'DRIVESP_cat_5_6_driver',
#  'DRIVESP_cat_GT7_driver',
 'DRIVESP_cat_No_Drive',
#  'ESR_cat_ArmedForces_at_work',
#  'ESR_cat_ArmedForces_not_work',
 'ESR_cat_Employed_at_work',
 'ESR_cat_Employed_not_work',
 'ESR_cat_Not_Labor',
#  'ESR_cat_Unemployed',
 'NAICSP_cat_ADM',
 'NAICSP_cat_AGR',
 'NAICSP_cat_CON',
 'NAICSP_cat_EDU',
 'NAICSP_cat_ENT1',
 'NAICSP_cat_ENT2',
 'NAICSP_cat_EXT',
 'NAICSP_cat_FIN1',
 'NAICSP_cat_FIN2',
 'NAICSP_cat_INF',
 'NAICSP_cat_MED',
 'NAICSP_cat_MFG1',
#  'NAICSP_cat_MFG2',
 'NAICSP_cat_MFG3',
#  'NAICSP_cat_No_Work',
 'NAICSP_cat_PRF1',
 'NAICSP_cat_PRF2',
 'NAICSP_cat_PRF3',
 'NAICSP_cat_RET1',
 'NAICSP_cat_RET2',
 'NAICSP_cat_SRV',
 'NAICSP_cat_UTL',
#  'NAICSP_cat_WHL',
 'POBP_cat_0',
#  'POBP_cat_1',
 'POWSP_cat_0',
#  'POWSP_cat_1',
#  'SCIENGRLP_cat_LT_Bachelors',
 'SCIENGRLP_cat_No',
#  'SCIENGRLP_cat_Yes',
#  'CITWP_0.0',
#  'CITWP_1.0',
 'COW_1.0',
 'COW_2.0',
 'COW_3.0',
 'COW_4.0',
 'COW_5.0',
 'COW_6.0',
 'COW_7.0',
#  'COW_8.0',
#  'COW_9.0',
#  'COW_10.0',
 'DDRS_1.0',
#  'DDRS_2.0',
 'DEAR_1',
#  'DEAR_2',
 'DPHY_1.0',
#  'DPHY_2.0',
 'DREM_1.0',
#  'DREM_2.0',
 'ENG_Not Well',
#  'ENG_Not at All',
#  'ENG_Only English',
 'ENG_Very Well',
 'ENG_Well',
#  'JWRIP_0.0',
#  'JWRIP_1.0',
#  'JWRIP_2.0',
#  'JWRIP_3.0',
#  'JWRIP_4.0',
#  'JWRIP_5.0',
#  'JWRIP_6.0',
#  'JWRIP_7.0',
#  'JWRIP_8.0',
#  'JWRIP_9.0',
#  'JWRIP_10.0',
#  'JWTR_1.0',
#  'JWTR_2.0',
 'JWTR_3.0',
 'JWTR_4.0',
 'JWTR_5.0',
 'JWTR_6.0',
 'JWTR_7.0',
#  'JWTR_8.0',
#  'JWTR_9.0',
 'JWTR_10.0',
#  'JWTR_11.0',
 'JWTR_12.0',
#  'JWTR_14.0',
 'MARHD_1.0',
#  'MARHD_2.0',
#  'MARHD_3.0',
#  'MARHM_1.0',
#  'MARHM_2.0',
#  'MARHM_3.0',
#  'MARHT_1.0',
#  'MARHT_2.0',
#  'MARHT_3.0',
#  'MIG_1.0',
 'MIG_2.0',
#  'MIG_3.0',
 'NWLK_1.0',
 'NWLK_2.0',
#  'NWLK_3.0',
#  'NWLK_4.0',
 'RELP_0',
 'RELP_1',
#  'RELP_2',
 'RELP_3',
 'RELP_4',
#  'RELP_5',
 'RELP_6',
 'RELP_7',
#  'RELP_8',
 'RELP_9',
 'RELP_10',
 'RELP_11',
#  'RELP_12',
#  'RELP_13',
#  'RELP_14',
 'RELP_15',
 'RELP_16',
 'RELP_17',
#  'SCH_1.0',
 'SCH_2.0',
#  'SCH_3.0',
#  'DECADE_0.0',
#  'DECADE_1.0',
 'DECADE_2.0',
 'DECADE_3.0',
#  'DECADE_4.0',
 'DECADE_5.0',
#  'DECADE_6.0',
 'DECADE_7.0',
 'DECADE_8.0',
 'DIS_Disabled',
#  'DIS_Not Disabled',
#  'ESP_0.0',
 'HISP_1',
 'HISP_2',
 'HISP_3',
 'HISP_4',
 'HISP_5',
#  'HISP_6',
 'HISP_7',
 'HISP_8',
 'HISP_9',
 'HISP_10',
 'HISP_11',
 'HISP_12',
 'HISP_13',
#  'HISP_14',
#  'HISP_15',
 'HISP_16',
 'HISP_17',
 'HISP_18',
 'HISP_19',
#  'HISP_20',
 'HISP_21',
 'HISP_22',
#  'HISP_23',
#  'HISP_24',
#  'INDP_ADM1',
#  'INDP_ADM2',
#  'INDP_ADM3',
#  'INDP_EDU',
#  'INDP_ENT1',
#  'INDP_ENT2',
#  'INDP_FIN1',
#  'INDP_FIN2',
#  'INDP_FIN3',
#  'INDP_FIN4',
#  'INDP_INF1',
#  'INDP_INF2',
#  'INDP_INF3',
#  'INDP_INF4',
#  'INDP_MED1',
#  'INDP_MED2',
#  'INDP_MED3',
#  'INDP_MED4',
#  'INDP_MFG1',
#  'INDP_MFG10',
#  'INDP_MFG11',
#  'INDP_MFG12',
#  'INDP_MFG13',
#  'INDP_MFG14',
#  'INDP_MFG15',
#  'INDP_MFG16',
#  'INDP_MFG17',
#  'INDP_MFG18',
#  'INDP_MFG19',
#  'INDP_MFG2',
#  'INDP_MFG20',
#  'INDP_MFG21',
#  'INDP_MFG22',
#  'INDP_MFG23',
#  'INDP_MFG24',
#  'INDP_MFG25',
#  'INDP_MFG26',
#  'INDP_MFG27',
#  'INDP_MFG28',
#  'INDP_MFG29',
#  'INDP_MFG3',
#  'INDP_MFG30',
#  'INDP_MFG4',
#  'INDP_MFG5',
#  'INDP_MFG6',
#  'INDP_MFG7',
#  'INDP_MFG8',
#  'INDP_MFG9',
#  'INDP_MIL1',
#  'INDP_MIL2',
#  'INDP_MIL3',
#  'INDP_None',
#  'INDP_PRF1',
#  'INDP_PRF2',
#  'INDP_PRF3',
#  'INDP_PRF4',
#  'INDP_PRF5',
#  'INDP_PRF6',
#  'INDP_RET1',
#  'INDP_RET10',
#  'INDP_RET11',
#  'INDP_RET12',
#  'INDP_RET2',
#  'INDP_RET3',
#  'INDP_RET4',
#  'INDP_RET5',
#  'INDP_RET6',
#  'INDP_RET7',
#  'INDP_RET8',
#  'INDP_RET9',
#  'INDP_SCA1',
#  'INDP_SCA2',
#  'INDP_SRV1',
#  'INDP_SRV2',
#  'INDP_SRV3',
#  'INDP_SRV4',
#  'INDP_SRV5',
#  'INDP_SRV6',
#  'INDP_TRN1',
#  'INDP_TRN2',
#  'INDP_TRN3',
#  'INDP_TRN4',
#  'INDP_WHL1',
#  'INDP_WHL2',
#  'INDP_WHL3',
#  'INDP_WHL4',
#  'INDP_WHL5',
#  'INDP_WHL6',
#  'MSP_1.0',
#  'MSP_2.0',
#  'MSP_3.0',
#  'MSP_4.0',
#  'MSP_5.0',
#  'MSP_6.0',
#  'NOP_0.0',
#  'OC_0.0',
#  'QTRBIR_1',
#  'QTRBIR_2',
#  'QTRBIR_3',
#  'QTRBIR_4',
#  'RACWHT_0',
#  'RACWHT_1',
#  'RC_0.0',
#  'RC_3.0',
#  'SCIENGP_0.0',
#  'SCIENGP_1.0',
#  'SCIENGP_2.0',
#  'VPS_16.0',
#  'WAOB_1',
#  'WAOB_2',
#  'WAOB_3',
#  'WAOB_4',
#  'WAOB_5',
#  'WAOB_6',
#  'WAOB_7',
#  'WAOB_8',
#  'CIT_STATUS_0',
#  'CIT_STATUS_1',
#  'BORN_CIT_0',
#  'BORN_CIT_1',
#  'RACES_American Indigenous',
#  'RACES_Asian',
#  'RACES_Black',
#  'RACES_More Than One Race',
#  'RACES_Other',
#  'RACES_White',
#  'MIL_STAT_Served',
#  'TYPE_1',
#  'TYPE_2',
#  'TYPE_3',
 'INDUSTRY_Administration',
 'INDUSTRY_Education',
 'INDUSTRY_Entertainment',
 'INDUSTRY_Finance',
 'INDUSTRY_Information',
 'INDUSTRY_Manufacturing',
 'INDUSTRY_Medical',
 'INDUSTRY_Military',
 'INDUSTRY_None',
 'INDUSTRY_Professional',
 'INDUSTRY_Retail',
#  'INDUSTRY_Service',
 'INDUSTRY_Social Service',
 'INDUSTRY_Transportation',
#  'INDUSTRY_Wholesale',
#  'SOCP_recat_BUS',
#  'SOCP_recat_CLN',
#  'SOCP_recat_CMM',
#  'SOCP_recat_CMS',
#  'SOCP_recat_EAT',
#  'SOCP_recat_EDU',
#  'SOCP_recat_ENG',
#  'SOCP_recat_ENT',
#  'SOCP_recat_EXT',
#  'SOCP_recat_FFF',
#  'SOCP_recat_FIN',
#  'SOCP_recat_HLS',
#  'SOCP_recat_LGL',
#  'SOCP_recat_MED',
#  'SOCP_recat_MGR',
#  'SOCP_recat_MIL',
#  'SOCP_recat_None',
#  'SOCP_recat_OFF',
#  'SOCP_recat_PRD',
#  'SOCP_recat_PRS',
#  'SOCP_recat_PRT',
#  'SOCP_recat_SAL',
#  'SOCP_recat_SCI',
#  'SOCP_recat_TRN'
                 ]

In [238]:
import statsmodels.api as sm
logit_model = sm.Logit(y_train, _X_train_merged[select_columns])
result = logit_model.fit()
result.summary()

         Current function value: 0.448612
         Iterations: 35


C:\Users\albert\Anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  HICOV   No. Observations:               242028
Model:                          Logit   Df Residuals:                   241851
Method:                           MLE   Df Model:                          176
Date:                Wed, 19 Aug 2020   Pseudo R-squ.:                  0.3528
Time:                        16:11:35   Log-Likelihood:            -1.0858e+05
converged:                      False   LL-Null:                   -1.6776e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
AGEP                          0.1574      0.013     11.671      0.000       0.131       0.184
OIP_cat                       0.0242      0.008      3.155      0.002       0.009       0.039
RETP_cat                      0.4616      0.023     20.464      0.000       0.417       0.506
SEMP_cat                     -0.0285      0.007     -3.896      0.000      -0.043      -0.014
SSIP_cat                      3.1348      0.245     12.799      0.000       2.655       3.615
JWDP_mins                     0.0348      0.009      3.747      0.000       0.017       0.053
PWGTP_y                       0.0044      0.005      0.811      0.417      -0.006       0.015
PAP                           0.0956      0.006     16.377      0.000       0.084       0.107
SSP                           1.7544      0.025     69.411      0.000       1.705       1.804
WAGP                          0.4191      0.015     28.000      0.000       0.390       0.448
OCCP                         -0.0499      0.008     -6.046      0.000      -0.066      -0.034
FINCP                         0.3837      0.011     35.722      0.000       0.363       0.405
MAR_YEARS                     0.3160      0.014     22.410      0.000       0.288       0.344
NP                            0.0831      0.007     12.182      0.000       0.070       0.096
CIT_1                         7.4564      0.522     14.295      0.000       6.434       8.479
CIT_2                         8.5340      0.535     15.964      0.000       7.486       9.582
CIT_3                         8.1835      0.537     15.240      0.000       7.131       9.236
CIT_4                         8.2921      0.539     15.387      0.000       7.236       9.348
CIT_5                         7.6165      0.539     14.130      0.000       6.560       8.673
DEYE_1                       -0.2641      0.056     -4.712      0.000      -0.374      -0.154
MAR_Divorced                 -0.3842      0.026    -14.963      0.000      -0.434      -0.334
MAR_Married                   0.2623      0.021     12.785      0.000       0.222       0.302
MAR_Separated                 0.0906      0.040      2.244      0.025       0.011       0.170
MAR_Widowed                   0.2554      0.047      5.437      0.000       0.163       0.347
SEX_Female                    0.1205      0.020      6.002      0.000       0.081       0.160
ANC_1                         0.0474      0.019      2.538      0.011       0.011       0.084
ANC_2                         0.1995      0.021      9.660      0.000       0.159       0.240
RACASN_0                     -0.2934      0.022    -13.084      0.000      -0.337      -0.249
RACBLK_0                      0.0474      0.018      2.659      0.008       0.012       0.082
RACNUM_1                     -0.1931      0.040     -4.794      0.000      -0.272      -0.114
RACNUM_4                     -1.5261      0.401     -3.807      0.000      -2.312      -0.741
RACPI_0                      -0.2575      0.1

In [188]:
# df['HISP'].value_counts()